**Download the data from below link**

[Data](https://drive.google.com/drive/folders/1OX-HMJ9g0abSPtivFOiiV54HXhIYsu_t?usp=sharing)

In [7]:
!gdown --folder https://drive.google.com/drive/folders/1OX-HMJ9g0abSPtivFOiiV54HXhIYsu_t -O /content/data --remaining-ok


Retrieving folder list
Retrieving folder 1O_Hy3EtSvaiZ5j-NVRd_vG5MIGBQ4iig testset
Processing file 1qP5OxjXwOh4Plyz5K0E59YucM-cYNwmb 0a3ff63cd2c98e2e2dd97f934e13ff8310e9cb26_1628837967.jpg
Processing file 1Xl9G0hfhpyxaeL4u1X65tOIERpk1bZt- 0a87e1764b165ffd47d14c8f2b89b5f68f03586f_1628887015.jpg
Processing file 1d4N8lj9uSn6s8jVqyux02L5nI-s7W-xv 0a256518b9e222d388136d87bcfef88a7e223eb9_1609830017.jpg
Processing file 15Npdc-mlJ5tkbpSLiWzEQyg5O_2KLEyI 0ad7821942377733c6045bff974f81a2835f24ce_1610947108.jpg
Processing file 1QyGGo29UUpOXdLSqAqVJvhK6oSnbEemA 0ada78dd267660a56b6dc3129450f2ca83672335_1623666166.jpg
Processing file 1icvDLDQDxhZ_MvugavI_BterAh4Af5Y4 0ae738c11e8dd454063393b6c25e66a176215bf9_1628967583.jpg
Processing file 1lQ2s-MSsAl3DBGfPB1Vg4_CDwyz55Y7q 0bafb5f0c2f04ebf9480c88b58310b651445c323_1629372782.jpg
Processing file 1q-_f2rrWUQCBRiKFAddy5MbJWaWCUysn 0c6157663a1ff9589087892c15638065f50494cb_1617718269.jpg
Processing file 1iykCfwbbmzGNj7T9g7gJPnFjYQu6d1tO 0cdbb5e9ecbddf0059b

In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1ZF_-tN-Ox6nBYpN0aaymPhTJK2pkGuaK -O /content/data/validset --remaining-ok

Retrieving folder list
Retrieving folder 1Q30C8syTsEy9UwWBa5m_18jtacCyJK91 1
Processing file 1XcbNtP0wFolX71jUl2Lk15cl1z_zpGeB 1b9cd9b6957cd80521f2532ff719cccff28c0087_1616068556.jpg
Processing file 1WiJ-RIwWHeKJYTQhQjbRNO1gHiPfNNLr 1fd985dc1dedf32893ec04d83a5ea7385071e8d4_1629878349.jpg
Processing file 1Xh7U1_QxwTleaML_SNgTARL1kWMZ5zBI 5deff859f35e3952df75d88e1d7d233353659776_1627977219.jpg
Processing file 1XrcwolT6Yw-F62rJYCY-msNTK-6jquHs 6e67757e8c418d5577eb6bcae0a45db4c81290b6_1615134709.jpg
Processing file 1Xp7xL0bPREBn7gakQSiMdiQYcY3a17al 6ead06b5d8df5f046ab63a97e5c8f2a3c3243809_1630218957.jpg
Processing file 1XvLnHlPk7fwV0V2yAqDskTWENLOA-1sY 8fdefa6388f76ae7b96a09d4d87c0d4dbbebd72c_1623586359.jpg
Processing file 1XIr-xpqvvEYFi6SUI1EYECRl2QgFj9gv 9a1ca8d931eaea5462523fa3f0f031bc8a5c0522_1629701711.jpg
Processing file 1XHGdAJ_WyfNp04eQ9VbB7qIZSDg_AO2i 29a7ca55a9cc7a59c2b6bd545349b417c8bccc2a_1611655239.jpg
Processing file 1Wi3Col4IS4KdCYDAAz5wIlazoDtXCV7q 378c3783a0400ff1505091c12

## Image classification using transfer learning

In [12]:
import pickle
import math, re, os
from os import path
import tensorflow as tf
import tensorflow.keras as keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import numpy as np
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import pandas as pd
import random
import shutil
import pathlib

In [13]:
EPOCHS = 50
BATCH_SIZE = 32

BASE_FOLDER = './data/'
WORKING_FOLDER = './'

CSV_LOCATION = f'{BASE_FOLDER}train.csv'
TRAINING_IMAGES_INPUT = f'{BASE_FOLDER}train_images/'
TEST_IMAGES_INPUT = f'{BASE_FOLDER}test_images/'
SUBMISSION_FILE = f'{WORKING_FOLDER}submission.csv'

In [14]:
def create_cnn_model():
  '''
  A function for creating cnn model from Mobilenetv2.
  customize the last dense layer with 10 classes. (This is a 10-class classification problem).
  load the imagenet weights od the mobilemetv2 network.
  don't use the official last layer of the net because it trained on a 1000-classes problem.
  You can add some new hidden layers using dense layer for more robust label learning.
  define the mode.
  freeze the official layers.
  compile model using adam optmizer and CE loss.
  report accuracy in your metrics.
  print model summary
  Hint:
    https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2/MobileNetV2
    https://keras.io/api/models/model/
  '''
  ##############################################
  ############# YOUR CODES GO HERE #############
  my_output = layers.Dense(10, activation='softmax')
  model = MobileNetV2(weights='imagenet')

  x = model.layers[-2].output
  x = layers.Dense(1024,activation='relu')(x)
  my_output = my_output(x)
  model = Model(inputs=model.input, outputs=my_output)
  for layer in model.layers[:-2]:
    layer.trainable = False


  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  print(model.summary())
  return model
  ##############################################

In [18]:
def create_callbacks():
    '''
    defining callbacks for having better control on model training process
      Early stopping: a technique for preventing from the overfitting.
        set patience = 8. monitor on val_loss
      ReduceLROnPlateau: reducing LR when the monitoring variable is not changing in a desired way.
        set patience = 8, mode = min, monitoring on val_loss and set the min learning rate = 0.001
      ModelCheckpoint: saving model best checkpoint according to the best value of the monitoring var.
        mpnitor on val_loss, define a path, just save the best checkpoint
      Hint: https://keras.io/api/callbacks/model_checkpoint/
            https://keras.io/api/callbacks/early_stopping/
            https://keras.io/api/callbacks/reduce_lr_on_plateau/
      retunr list of the callbacks
    '''
    early_stopping = EarlyStopping(patience=8, monitor='val_loss', verbose=1)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', min_lr=0.001,
                                  patience=8, mode='min',
                                  verbose=1)

    model_checkpoint = ModelCheckpoint(monitor='val_loss',
                                       filepath='./my_mobilenet_model.h5',
                                       save_best_only=True,
                                       verbose=1)

    callbacks = [
        early_stopping,
        reduce_lr,
        model_checkpoint
    ]

    return callbacks

In [19]:
def train_model_naive_split():
    '''
    read the data and train model.
    we want to read data using ImageDataGenerator.
    this generator, expects the training or validation data stored in a directory, images belongs to a specific class
      be under a directory called as the class name or id, one directory per class.
      like root_path:
              |
              -----class 1:
                    |
                    ------- images of class 1
              -----class 2:
                    |
                    ------- images of class 2
              .....
    you can add some preprocessing steps to it.
    this can read the vlidation dataset in a same way.
    define rescale argument in it for normalizing images between 0-1 (so it should be the 1. / 255).
    in flow_from_directory method give the method root_oath of training and validation dir respectively for train_gen and val_gen.
    set target_size = 224 * 224 (resizing step)
    set the batch_size you want.
    set class_mode = 'categorical' for better management the labels.
    create the model.
    fit model on train_itr. give it the val_itr for validation, create callbacks and pass it through the fit function.
    Hint:
      https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
    return:
      history of training process
    '''

    train_gen = ImageDataGenerator(
        rescale=1. / 255,
    )

    train_iterator = train_gen.flow_from_directory('/content/data/trainset/',
                                                   target_size=(224, 224),
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical')
    valid_gen = ImageDataGenerator(
        rescale=1. / 255,
    )

    valid_iterator = valid_gen.flow_from_directory('/content/data/validset/',
                                                   target_size=(224, 224),
                                                   batch_size=BATCH_SIZE,
                                                   class_mode='categorical')

    model = create_cnn_model()

    history = model.fit(train_iterator,
                        validation_data=valid_iterator,
                        epochs=200,
                        callbacks=create_callbacks())

    return history

In [21]:
## run training
history = train_model_naive_split()

Found 398 images belonging to 8 classes.
Found 0 images belonging to 1 classes.
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
            

InvalidArgumentError: ignored

In [ ]:
## plot the val_loss, train_loss, val_acc and train_acc plots

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy curves
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [23]:
import sys
import glob
import tensorflow as tf
from PIL import Image
import cv2
import numpy as np

In [25]:
## glob test images for inference
test_imhs_path = "/content/data/testset/"
images = glob.glob(test_imhs_path + "*.jpg")

In [ ]:
## load model from stored checkpoints

model = model = tf.keras.models.load_model('/content/my_mobilenet_model.h5')

In [ ]:
## We want to store predicted data in a .csv file
import csv
f = open('/content/output.csv', 'w')
writer = csv.writer(f)
writer.writerow(["name", "predicted"])
f.close()

In [ ]:
## iterate through test images, read them using cv2, normalize them, resize to target size then predict class of them using model.
## model will return the probability for each class, find the best class for each image and write in csv file created above.

for img in images:
  original_image = cv2.imread(img)
  image_data = cv2.resize(original_image, (224, 224))
  image_data = image_data / 255.
  image_data = image_data.astype(np.float32)

  pred = model.predict(np.reshape(image_data, (-1, 224, 224, 3)))
  res = np.argmax(pred)+1
  img_name = img.split('/')[-1]

  f = open('/content/output.csv', 'a')
  writer = csv.writer(f)
  writer.writerow([img_name, res])
  f.close()


In [ ]:
import pandas as pd

df = pd.read_csv("/content/output.csv")

In [ ]:
!head /content/output.csv